# 计算广告

> 15 年看《计算广告》时，虽然在广告行业，担也是云里雾里；19 年再看才恍然大悟。

通篇最核心公式：

$$
r(a, u, c) = \mu(a, u, c) * v(a, u, c)
$$

## 9. 广告技术概览

1. 流式计算 (flume, storm)
2. Hadoop

- [ ] PAXOS
- [ ] ZAB

GD (Guranteed Delivery): 流量控制和频次控制；

- [ ] 一个想法, 在做单次检查频控的时候为什么不用 Bloom Filter 那？如果用 Bloom Filter 的话，怎么用在分布式的环境中那？毕业内存级的 Filter 不太可能吧？ 所以说分布式系统很重要, Redis 怎么分布式那？

#### 11.2.2 频次控制

在传递的电话广告中存在着 "three hit theory"。但是在现代的计算广告中并不是这样的，实际中，

- 首先，广告的频次与广告效果（eCPM) 的关系是负相关的，即广告效果随着频次的上升呈现单调的下降趋势；而并非在第三次时达到最佳；
- 其次，频次较高的广告展示效果很差，因为，没有足够多的广告主数量，整体的广告效果会受到相当大的限制；（没有足够多的广告主，不要做商业化）

在竞价广告中，频次可以作为一个特征来应用。合约广告中品牌广告主可能会主动要求频次，而在 CPC 结算的竞价广告中，可以将频次作为 CTR 预估的特征之一，从而隐式的对广告的重复展示进行控制。（模型可以根据频次隐式的提高或降低点击率，频次是一个实时特征）。

## 12. 受众定向核心技术

广告两个最关键的技术：广告检索和广告排序。


## 13. 竞价广告核心技术

### 13.1 竞价广告计价算法（广告排序与计费）

1. $eCPM = ctr * bid$, 排序依据；
2. 按第二高价计费；
3. 需要设置最低出价，MRP（Market Reserve Price, 市场保留价）


对于一个 CPC 结算的竞价广告系统，需要先得到广告候选集合，并计算每个候选的点击率，这对应于竞价广告两个最关键的计算问题：广告检索与广告排序。

### 13.2 搜索广告系统

$$
\max_{a_1,\cdots, T} \sum_{i=1}^{T}\{\mu(a_i, c_i) \cdot bid_{CPC}(a_i)\}
$$

对每次展示的各个候选，根据查询估计其点击率 $\mu$, 并乘以广告主的点击出价得到 eCPM, 再按此排序即可。

#### 13.2.2 广告放置 （北区、东区）

一般广告放置指的是搜索引擎广告中确定北区和东区的广告条数的问题。考虑到用户体验需要对北区广告的数据进行限制，所以这是一个典型的带约束条件的优化问题：**约束是系统在一段时间内整体的北区广告条数，而优化的目标则是搜索广告系统的整体营收。** 

- [ ] BTW, 其实，很多策略都会有这个约束，关键是要找到最终的优化目标，以及相应的约束条件。

在进行广告放置之前的排序过程中，比较的都是单条广告的 eCPM，而此处优化的是一组广告，并且需要考虑位置因素，因此可以用下式来表达：

$$
\max\sum_{i=1}^{T}\Big\{\sum_{s=1}^{n_i}r(a_{is}, c_i, N_s) + \sum_{s=n_i + 1}^{n_i + e_i} r(a_{is}, c_i, E_{s - n_i}) \Big\} \\
\sum_{i=1}^{T} n_i \le TC
$$

- $n_i$ : North 第 i 次展示北区的广告条数
- $e_i$ : East 第 i 将展示东区的广告条数
- $N_s$ : 北区第 s 个位置
- $E_s$ : 东区第 s 个位置
- $C$ : 为北区的平均广告条数上限；实际上在排序阶段，这个参数等于 $N_1$ 来处理。

**问题**: 调整北区广告准入的一些指标，如 MRP, 相关性，质量度等，都会影响此问题的解。其实，广告的位置具有的天然的点击率，以及 MRP 高怎么办？

实际中，在排序阶段，我们都是按 $N_1$ 这个位置来处理。而对于不同的用户动态的来调整北区广告的条目数，可以使得北区平均广告条目数相同的约束下，整体系统的营收有显著的提高:

$$
r'(a_{is}, u_i, c_i, N_s) = r(a_{is}, c_i, N_s) {\overline{\mu(u_i)}}/{\overline{\mu}}
$$

- ${\overline{\mu(u_i)}}$: 用户 $\mu_i$ 对北区广告的平均点击率；
- $\overline{\mu}$: 所有用户对北区广告的平均点击率。

### 13.3 广告网络

广告网络的优化目标可以在 2.2 的基础上，用下面的式子表达:


$$
\max_{a_1,\cdots, T} \sum_{i=1}^{T}\{\mu(a_i, u_i, c_i) \cdot bid_{CPC}(a_i)\}
$$


广告网络的典型系统架构为图 13-2，其中广告的投放决策流程为: 服务器接收前端用户请求-->生成 u 标签和 c 标签 --> 找出符合要求的广告候选集合--> CTR 预估 --> 计算 eCPM --> 根据 eCPM 排序选出赢得竞价的广告-->并返回前端完成投放。

在这个投放决策流程中，需要：

1. CTR 预估；
2. 准实时的计费；
3. 点击反作弊；
4. 用户行为尽快反馈到广告决策中；a) 短时用户标签；b)短时动态特征；技术:流式计算(spark/strom).

### 13.4 广告检索

倒排索引是搜索引擎的关键技术。广告检索也使用倒排索引这样的框架。